In [1]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import scipy
import corner
import pymc3 as pm
import seaborn as sns
import os
import time


import theano
import theano.tensor as T
from theano.ifelse import ifelse

import sys
sys.path.append("../theano_ops")
sys.path.append("../codebase")
from data_preprocessing_ogle import process_data
from plotting_utils import plot_data
from theano_ops.celerite.factor import FactorOp
from theano_ops.celerite.solve import SolveOp
from theano_ops.celerite import terms
from theano_ops.celerite.celerite import log_likelihood

from scipy.special import gamma
from scipy.stats import invgamma
from scipy.optimize import fsolve

%matplotlib inline

In [2]:
# DFM's pymc3 hack for estimating off-diagonal mass-matrix terms in NUTS during
# burn-in period
from pymc3.step_methods.hmc.quadpotential import QuadPotentialFull
def get_step_for_trace(trace=None, model=None,
                       regular_window=5, regular_variance=1e-3,
                       **kwargs):
    model = pm.modelcontext(model)
    
    # If not given, use the trivial metric
    if trace is None:
        potential = QuadPotentialFull(np.eye(model.ndim))
        return pm.NUTS(potential=potential, **kwargs)
        
    # Loop over samples and convert to the relevant parameter space;
    # I'm sure that there's an easier way to do this, but I don't know
    # how to make something work in general...
    samples = np.empty((len(trace) * trace.nchains, model.ndim))
    i = 0
    for chain in trace._straces.values():
        for p in chain:
            samples[i] = model.bijection.map(p)
            i += 1
    
    # Compute the sample covariance
    cov = np.cov(samples, rowvar=0)
    
    # Stan uses a regularized estimator for the covariance matrix to
    # be less sensitive to numerical issues for large parameter spaces.
    # In the test case for this blog post, this isn't necessary and it
    # actually makes the performance worse so I'll disable it, but I
    # wanted to include the implementation here for completeness
    N = len(samples)
    cov = cov * N / (N + regular_window)
    cov[np.diag_indices_from(cov)] += \
        regular_variance * regular_window / (N + regular_window)
    
    # Use the sample covariance as the inverse metric
    potential = QuadPotentialFull(cov)
    return pm.NUTS(potential=potential, **kwargs)

def solve_for_invgamma_params(params, x_min, x_max):
    """Returns parameters of an inverse gamma distribution p(x) such that 
    0.1% of total prob. mass is assigned to values of x < x_min and 
    1% of total prob. masss  to values greater than x_max."""
    def inverse_gamma_cdf(x, alpha, beta):
        return invgamma.cdf(x, alpha, scale=beta)

    lower_mass = 0.001
    upper_mass = 0.99

    # Trial parameters
    alpha, beta = params

    # Equation for the roots defining params which satisfy the constraint
    return (inverse_gamma_cdf(2*x_min, alpha, beta) - \
    lower_mass, inverse_gamma_cdf(x_max, alpha, beta) - upper_mass)


def fit_pymc3_model(t, F, sigF):
    model = pm.Model()

    # SPECIFICATION OF PRIORS
    # Compute parameters for the prior on GP hyperparameters
    invgamma_a, invgamma_b =  fsolve(solve_for_invgamma_params, (0.1, 0.1), 
        (np.median(np.diff(t)), t[-1] - t[0]))

    strt = time.time()
    with model:    
        # Priors for GP hyperparameters
        def ln_rho_prior(ln_rho):
            lnpdf_lninvgamma = lambda  x, a, b: np.log(x) + a*np.log(b) -\
                 (a + 1)*np.log(x) - b/x - np.log(gamma(a)) 

            res = lnpdf_lninvgamma(np.exp(ln_rho), invgamma_a, invgamma_b)
            return T.cast(res, 'float64')

        ln_rho = pm.DensityDist('ln_rho', ln_rho_prior, testval = 0.6)

        def ln_sigma_prior(ln_sigma):
            sigma = np.exp(ln_sigma)
            res = np.log(sigma) - sigma**2/3.**2
            return T.cast(res, 'float64')

        ln_sigma = pm.DensityDist('ln_sigma', ln_sigma_prior, testval=2.)

#         u_K = pm.Uniform('u_K', -1., 1.)

#         K = ifelse(u_K < 0., T.cast(1., 'float64'), 1. - T.log(1. - u_K))

        # CALCULATE LIKELIHOOD
        def custom_log_likelihood(t, F, sigF):
            mean_function = 0.
            kernel = terms.Matern32Term(sigma=ln_sigma, rho=ln_rho)
            loglike = log_likelihood(kernel, mean_function,
                sigF, t, F)

            return loglike 

        logl = pm.DensityDist('logl', custom_log_likelihood, 
            observed={'t': t, 'F':F, 'sigF': sigF})

        for RV in model.basic_RVs:
            print(RV.name, RV.logp(model.test_point))

        # Fit model with NUTS
        #trace = pm.sample(2000, tune=2000, nuts_kwargs=dict(target_accept=.95),
        #    start=start)

        # DFM's optimized sampling procedure
        start = None
        burnin_trace = None
        for steps in n_window:
            step = get_step_for_trace(burnin_trace, regular_window=0)
            burnin_trace = pm.sample(
                start=start, tune=steps, draws=2, step=step,
                compute_convergence_checks=False, discard_tuned_samples=False)
            start = [t[-1] for t in burnin_trace._straces.values()]

        step = get_step_for_trace(burnin_trace, regular_window=0)
        dense_trace = pm.sample(draws=5000, tune=n_burn, step=step, start=start)
        factor = 5000 / (5000 + np.sum(n_window+2) + n_burn)
        dense_time = factor * (time.time() - strt)

    return dense_trace, dense_time

In [ ]:
events = [] # event names
lightcurves = [] # data for each event
 
i = 0
n_events = 1
for entry in os.scandir('/home/star/fb90/data/OGLE_ews/2017/'):
    if entry.is_dir() and (i < n_events):
        events.append(entry.name)
        photometry = np.genfromtxt(entry.path + '/phot.dat', usecols=(0,1,2))
        lightcurves.append(photometry)
        i = i + 1
        
print("Loaded events:", events)

# Define a tuning schedule for HMC
n_start = 25
n_burn = 500
n_tune = 5000
n_window = n_start * 2 ** np.arange(np.floor(np.log2((n_tune - n_burn) / n_start)))
n_window = np.append(n_window, n_tune - n_burn - np.sum(n_window))
n_window = n_window.astype(int)
np.random.seed(42)


for event_index, lightcurve in enumerate(lightcurves):
    # Pre process the data
    t, F, sigF = process_data(lightcurve[:, 0], lightcurve[:, 1], 
        lightcurve[:, 2], standardize=True)
    
    t = t[:500]
    F = F[:500]
    sigF = sigF[:500]

    fig, ax = plt.subplots(figsize=(20, 6))
    fig2, ax2 = plt.subplots(figsize=(20, 6))

    plot_data(ax, t, F, sigF)

    # Fit pymc3 model
    dense_trace, dense_time = fit_pymc3_model(t, F, sigF)
    stats = pm.summary(dense_trace)
    dense_time_per_eff = dense_time / stats.n_eff.min()
    print("time per effective sample: {0:.5f} ms".format(dense_time_per_eff * 1000))

    fig, ax = plt.subplots(5, 2 ,figsize=(10,10))
    _ = pm.traceplot(dense_trace, ax=ax2)

Loaded events: ['blg-0001']
ln_rho -1.3731855828410553
ln_sigma -4.066461114793804
logl -439.0106413406179


Only 2 samples in chain.
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [ln_sigma, ln_rho]
Sampling 4 chains:  27%|██▋       | 29/108 [00:19<00:00, 154.33draws/s]